# Sujet 2: Modélisation des interdépendances

**Quel est l’impact d’une baisse de la capacité de LNG sur le système en général et en particulier sur
le système électrique ?**

Pour aller plus loin : modéliser le stockage gaz

**Authors**: Lucas DEFRANCE, Quentin DEHAENE, Jules PAINDESSOUS--POETE

## Imports

In [1]:
# Necessary imports
using JuMP
using XLSX
using DataFrames
#use the solver you want
using HiGHS

In [2]:
# Define global variables
data_file_zootopia = "data/TP_empilement.xlsx";

In [3]:
# We choose

In [9]:
nb_week_year = 52
for w in 0:nb_week_year
    # We start the simulation at Tmin and end it at Tmax
    Tmin = w*24*7
    Tmax = Tmin + 24*7-1; #optimization by one week intervals
    Tp = 24*7;
    
    #data for load and fatal generation
    load = XLSX.readdata(data_file_zootopia, "TP", "C$(10+Tmin):C$(Tmax + 10)");
    wind = XLSX.readdata(data_file_zootopia, "TP", "D$(10+Tmin):D$(Tmax + 10)");
    solar = XLSX.readdata(data_file_zootopia, "TP", "E$(10+Tmin):E$(Tmax + 10)");
    hydro_fatal = XLSX.readdata(data_file_zootopia, "TP", "F$(10+Tmin):F$(Tmax + 10)");
    thermal_fatal = XLSX.readdata(data_file_zootopia, "TP", "G$(10+Tmin):G$(Tmax + 10)");
    #total of RES
    Pres = wind + solar + hydro_fatal + thermal_fatal;


    #############################
    #create the optimization model
    #############################
    model = Model(HiGHS.Optimizer)

    #############################
    #define the variables
    #############################
    #thermal generation variables
    
    @variable(model, Pth[1:Tp,1:Nth] >= 0)
    @variable(model, UCth[1:Tp,1:Nth], Bin)
    @variable(model, UPth[1:Tp,1:Nth], Bin)
    @variable(model, DOth[1:Tp,1:Nth], Bin)
    #hydro generation variables
    @variable(model, Phy[1:Tp,1:Nhy] >= 0)
    #unsupplied energy variables
    @variable(model, Puns[1:Tp] >= 0)
    #in excess energy variables
    @variable(model, Pexc[1:Tp] >= 0)
    #weekly STEP variables
    @variable(model, Pcharge_STEP[1:Tp] >= 0)
    @variable(model, Pdecharge_STEP[1:Tp] >= 0)
    @variable(model, stock_STEP[1:Tp] >= 0)

    #############################
    #define the objective function
    #############################
    @objective(model, Min, sum(Pth.*cth)+sum(Phy.*chy)+Puns'cuns+Pexc'cexc)


    #############################
    #define the constraints
    #############################
    #balance constraint
    @constraint(model, balance[t in 1:Tp], sum(Pth[t,g] for g in 1:Nth) + sum(Phy[t,h] for h in 1:Nhy) + Pres[t] + Puns[t] - load[t] - Pexc[t] - Pcharge_STEP[t] + Pdecharge_STEP[t]== 0)
    #thermal unit Pmax constraints
    @constraint(model, max_th[t in 1:Tp, g in 1:Nth], Pth[t,g] <= Pmax_th[g]*UCth[t,g])
    #thermal unit Pmin constraints
    @constraint(model, min_th[t in 1:Tp, g in 1:Nth], Pmin_th[g]*UCth[t,g] <= Pth[t,g])


    #thermal unit Dmin constraints
    for g in 1:Nth
        if (dmin_th[g] > 1)
            @constraint(model, [t in 2:Tp], UCth[t,g]-UCth[t-1,g]==UPth[t,g]-DOth[t,g],  base_name = "fct_th_$g")
            @constraint(model, [t in 1:Tp], UPth[t]+DOth[t]<=1,  base_name = "UPDOth_$g")
            if (w==0)
                @constraint(model, UPth[1,g]==0,  base_name = "iniUPth_$g")
                @constraint(model, DOth[1,g]==0,  base_name = "iniDOth_$g")
            else
                @constraint(model, UCth[1,g]-UC_continu=UPth[1,g]-DOth[1,g],  base_name = "fct_th_0$g")
            end
                


            @constraint(model, [t in dmin_th[g]:Tmax], UCth[t,g] >= sum(UPth[i,g] for i in (t-dmin_th[g]+1):t),  base_name = "dminUPth_$g")
            @constraint(model, [t in dmin_th[g]:Tmax], UCth[t,g] <= 1 - sum(DOth[i,g] for i in (t-dmin_th[g]+1):t),  base_name = "dminDOth_$g")
            @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t), base_name = "dminUPth_$(g)_init")
            @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t), base_name = "dminDOth_$(g)_init")
        end
    end 

    
    #hydro unit constraints
    @constraint(model, unit_bounds_hy[t in 1:Tp, h in 1:Nhy], Pmin_hy[h] <= Phy[t,h] <= Pmax_hy[h])
    #hydro stock constraint
    @constraint(model, stock_bounds_hy[h in 1:Nhy], sum(Phy[t,h] for t in 1:Tp) <= weekly_mean_e_hy[h] )

    #weekly STEP
    @constraint(model, max_Pcharge_STEP[t in 1:Tp],Pcharge_STEP[t] <= Pmax_STEP)
    @constraint(model, max_Pdecharge_STEP[t in 1:Tp],Pdecharge_STEP[t] <= Pmax_STEP)

    @constraint(model, stock_STEP_max[t in 1:Tp], stock_STEP[t] <= max_stock_STEP)
    @constraint(model, stock_STEP_evo[t in 2:Tp],stock_STEP[t] == stock_STEP[t-1] + Pcharge_STEP[t]*rSTEP - Pdecharge_STEP[t])
    if (w==0)
        @constraint(model,stock_STEP[1] == 0.5*max_stock_STEP)
    else
        @constraint(model,stock_STEP[1] == stock_STEP_continu + Pcharge_STEP[1]*rSTEP - Pdecharge_STEP[1])
    end




    #no need to print the model when it is too big
    #solve the model
    optimize!(model)


    UC_continu=UCth[Tp,1:Nth];
    stock_STEP_continu=stock_STEP[Tp];
    UP_continu=UPth[Tp-24:Tp,1:Nth];
    DO_continu=DOth[Tp-24:Tp,1:Nth];


end

168168168168

LoadError: InterruptException: